In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModel, AutoTokenizer 


In [3]:
model_name = "urduhack/roberta-urdu-small" 


model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

inputs = tokenizer("پاکستان کا دارالحکومت", return_tensors="pt")

outputs = model(**inputs)

/Users/faizanfaisal/anaconda3/envs/urdu-rag/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Mean pooling to get a single sentence embedding
sentence_embedding = outputs.last_hidden_state.mean(dim=1)
sentence_embedding[0]

tensor([-2.6271e-02,  6.9214e-02,  3.9749e-01, -5.7898e-01, -2.5997e-01,
         1.0924e-01,  5.4641e-01, -4.3341e-01,  5.6611e-02, -3.9940e-01,
        -1.5064e-01, -3.9552e-01,  5.2472e-01, -5.3057e-01, -1.0961e-01,
         7.9295e-01,  4.0281e-01, -9.5279e-02, -5.0134e-01, -6.7030e-01,
        -4.3296e-01, -2.5009e-01, -1.7574e-01, -2.1898e-01,  5.0585e-01,
         1.5430e-01, -7.2050e-01, -2.8741e-01,  6.5067e-02,  5.9066e-01,
         4.5272e-01,  1.3131e-01,  3.9579e-01,  5.0163e-01, -8.4291e-01,
         4.4362e-01, -1.1749e+00,  2.5488e-01, -5.2431e-01, -5.6871e-01,
         3.1260e-01, -1.5233e+00,  1.8554e-01,  4.9806e-01, -5.9515e-01,
         8.9338e-01,  4.5734e-01,  3.4641e-01, -3.2990e-02,  7.7993e-01,
         7.7631e-02,  1.5489e-01,  2.0793e-02,  3.4015e-01, -5.4190e-01,
         1.1469e+00, -6.3744e-01, -8.7200e-01,  2.7037e+00,  6.8973e-01,
        -1.0472e+00, -2.8600e-01,  4.7260e-01, -1.4766e-01,  3.8357e-01,
        -7.9783e-01,  5.3591e-01,  2.3764e-01, -1.2

In [5]:
import json

# Path to your JSON file
file_path = '../../data/english_to_urdu_translations/openai_translations.json'

with open(file_path, 'r', encoding='utf-8') as file:
    dataset = json.load(file)

In [6]:
urdu_questions = [i['urdu_question'] for i in dataset]
urdu_answers = [i['urdu_answer'] for i in dataset]

In [31]:
from tqdm import tqdm
import numpy as np

def get_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    sentence_embedding = outputs.last_hidden_state.mean(dim=1)
    return sentence_embedding[0].detach().numpy()  # Convert to numpy array for further use

def generate_embeddings(urdu_answers, tokenizer, model):
    embeddings = []
    for answer in tqdm(urdu_answers, desc="Generating embeddings"):
        embedding = get_embedding(answer, tokenizer, model)
        embeddings.append(embedding)
    return embeddings

def get_nearest_answer(question_embedding, answer_embeddings, top_k=5):
    similarities = cosine_similarity([question_embedding], answer_embeddings)
    best_match_index = np.argsort(similarities)[0][::-1][:top_k]
    return best_match_index

In [8]:
answer_embeddings = generate_embeddings(urdu_answers, tokenizer, model)
question_embeddings = generate_embeddings(urdu_questions, tokenizer, model)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Generating embeddings: 100%|██████████| 621/621 [00:41<00:00, 14.95it/s]


In [5]:
def calculate_score(number, array, top_k=5):
    try:
        position = list(array).index(number)
        if position < top_k:
            return 1
        else:
            return 0
    except ValueError:
        return 0

In [70]:
def evaluate_accuracy(urdu_questions, urdu_answers, question_embeddings, answer_embeddings, top_k=1, weighted=False):
    correct_retrievals = 0
    
    for i, question_embedding in enumerate(tqdm(question_embeddings, desc="Evaluating accuracy")):
        best_match_indices = get_nearest_answer(question_embedding, answer_embeddings, top_k)
        correct_retrievals += calculate_score(i, best_match_indices, top_k)
    
    # Calculate accuracy as the percentage of correct retrievals
    accuracy = (correct_retrievals / len(urdu_questions)) * 100
    return accuracy


In [11]:
for i in range(5):
    top_k = i + 1
    accuracy = evaluate_accuracy(urdu_questions, urdu_answers, question_embeddings, answer_embeddings, top_k)
    print(f"Top-{top_k} Accuracy (Weighted): {accuracy:.2f}%")

Evaluating accuracy: 100%|██████████| 621/621 [00:02<00:00, 239.76it/s]


Top-1 Accuracy (Weighted): 61.84%


Evaluating accuracy: 100%|██████████| 621/621 [00:02<00:00, 290.96it/s]


Top-2 Accuracy (Weighted): 69.73%


Evaluating accuracy: 100%|██████████| 621/621 [00:02<00:00, 286.48it/s]


Top-3 Accuracy (Weighted): 73.91%


Evaluating accuracy: 100%|██████████| 621/621 [00:02<00:00, 265.24it/s]


Top-4 Accuracy (Weighted): 75.52%


Evaluating accuracy: 100%|██████████| 621/621 [00:02<00:00, 282.08it/s]

Top-5 Accuracy (Weighted): 77.78%


In [7]:
from datasets import load_from_disk

In [13]:
dataset_dict = load_from_disk('../../data/complete_dataset')

In [14]:
all_contexts = {}

for i in dataset_dict['train']:
    all_contexts[i['context_index']] = i['context_urdu']


for i in dataset_dict['validation']:
    all_contexts[i['context_index']] = i['context_urdu']

In [18]:
sorted_article_indices = sorted(all_contexts.keys())
sorted_articles = [all_contexts[i] for i in sorted_article_indices]

In [41]:
sorted_articles

['۱۔ مملکت پاکستان ایک وفاقی جمہوریہ ہوگی جس کا نام اسلامی جمہوریہ پاکستان ہوگا جسے بعد ازیں پاکستان کہا جائے گا۔\n\n۲۔ پاکستان کے علاقے مندرجہ ذیل پر مشتمل ہوں گے:\n\n- صوبہ جات: بلوچستان، خیبر پختونخوا، پنجاب اور سندھ؛\n- دارالحکومت اسلام آباد کا علاقہ جس کا حوالہ بعد ازیں وفاقی دارالحکومت کے طور پر دیا گیا ہے؛\n- ایسی ریاستیں اور علاقے جو الحاق کے ذریعے یا کسی اور طریقے سے پاکستان میں شامل ہیں یا ہو جائیں۔\n\n۳۔ مجلس شوریٰ (پارلیمنٹ) بذریعہ قانون وفاق میں نئی ریاستوں یا علاقوں کو ایسی قیود و شرائط پر داخل کر سکے گی جو وہ مناسب سمجھے۔',
 'اسلام ریاست کا مذہب ہوگا۔',
 'اسلام پاکستان کا مملکتی مذہب ہوگا۔\n\n۲۲ الف۔ ضمیمہ میں نقل کردہ قرارداد مقاصد میں بیان کردہ اصول اور احکام کو بذریعہ ہذا قرارداد مقاصد مستقل دستور کا مستقل حصہ قرار دیا جاتا ہے اور وہ بحسبہ مؤثر ہوں گے۔',
 'استحصال کا خاتمہ\n\nریاست استحصال کی تمام اقسام کے خاتمے اور اس بنیادی اصول کی تدریجی تکمیل کو یقینی بنائے گی کہ ہر کسی سے اس کی اہلیت کے مطابق کام لیا جائے گا اور ہر کسی کو اس کے کام کے مطابق معاوضہ دیا جائے۔',
 'ا

In [33]:
context_embeddings = generate_embeddings(sorted_articles, tokenizer, model)

Generating embeddings: 100%|██████████| 305/305 [00:31<00:00,  9.59it/s]


In [37]:
urdu_qs = dataset_dict['validation']['question_urdu']
asnwer_context_indices = dataset_dict['validation']['context_index']
display(urdu_qs[:3])
display(asnwer_context_indices[:3])

['اسلامی کونسل کے رکن کی مدت کار کتنی ہوتی ہے؟',
 'پاکستان میں یوم آغاز سے قبل کن علاقوں کو قبائلی علاقے جات سمجھا جاتا تھا؟',
 'کیا کسی عدالت عالیہ کے جج کا اس کی رضامندی کے بغیر کسی دوسری عدالت عالیہ میں تبادلہ کیا جا سکتا ہے؟']

[253, 271, 209]

In [49]:
question_embeddings = generate_embeddings(urdu_qs, tokenizer, model)

Generating embeddings: 100%|██████████| 124/124 [00:05<00:00, 22.49it/s]


In [71]:
evaluate_accuracy(urdu_qs, asnwer_context_indices, question_embeddings, context_embeddings, top_k=5)

Evaluating accuracy: 100%|██████████| 124/124 [00:00<00:00, 815.95it/s]


3.225806451612903